In [ ]:

SYSTEM_PROMPT = """You are an enthusiastic yet concise running coach.
Your role is to deliver a real-time update in 1–2 sentences (≤ 35 words).
Speak to the runner in the second person singular. Never invent numbers.
"""

USER_TEMPLATE = """
Current data
------------
Address              : {Address}
Distance covered     : {Distance_done_km:.1f} km
Distance remaining   : {Distance_remaining_km:.1f} km
Current pace         : {Actual_pace} min/km
Target pace          : {Target_pace} min/km
Heart rate           : {Heart_rate_bpm} bpm
"""

def build_prompt(row):
    return SYSTEM_PROMPT + USER_TEMPLATE.format(
        Address=row["Address"],
        Distance_done_km=row["Distance_done_m"] / 1000,
        Distance_remaining_km=row["Distance_remaining_m"] / 1000,
        Actual_pace=row["Actual_pace_min_per_km"],
        Target_pace=row["Target_pace_min_per_km"],
        Heart_rate_bpm=row["Heart_rate_bpm"],
    )

import subprocess
from typing import Optional, List

class OllamaLLM:
    """Wrapper synchrone autour de `ollama run`."""
    def __init__(self, model_name="gemma:latest"): 
        self.model_name = model_name

    def __call__(self, prompt, stop=None):
        import subprocess
        res = subprocess.run(
            ["ollama", "run", self.model_name],
            input=prompt,
            capture_output=True,
            text=True,
            timeout=180,
        )
        if res.returncode != 0:
            raise RuntimeError(res.stderr.strip())
        out = res.stdout.strip()
        out = out.split("\n")[0]
        if stop:
            for tok in stop:
                out = out.split(tok)[0]
        return out
import pandas as pd, pathlib, sys

df = pd.read_excel("../data/runningluc.xlsx", engine="openpyxl")


llm = OllamaLLM("gemma:latest")   
for _, row in df.iloc[::5].iterrows():
    prompt  = build_prompt(row)
    message = llm(prompt)         
    km      = row["Distance_done_m"] / 1000
    print(f"[{km:4.1f} km] {message}")

[ 0.2 km] Pace slightly slower, but keep pushing! 5.6 km/h is within reach.
[ 1.2 km] Keep pushing! Your pace is strong, but a bit faster pace will get you closer to your target.
[ 2.2 km] Keep pushing! Your pace is close to the target. Focus and you'll reach the finish line in no time.
[ 3.2 km] Keep pushing! Your pace is great, focus on maintaining it for the remaining distance.
[ 4.2 km] Eyes on the target! Maintain that smooth stride, and you'll reach the finish in under 13 minutes.
[ 5.2 km] Keep pushing! Your current pace is good, but let's aim for that 5.17 min/km to maximize your training.
[ 6.2 km] Pace looking good! Keep pushing for that 5.17 target. 💪
[ 7.2 km] Eyes on the target! Your pace is close, keep pushing for the final kilometer.
[ 8.2 km] Eyes on the prize! You're in the final stretch. Keep pushing for a 5.67km pace.
[ 9.2 km] Keep pushing! You're in the final stretch and can reach your target pace.
[10.2 km] Eyes on the prize! You're nearing the finish. Keep that p